In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-3.5-turbo", temperature=0)
response = llm.invoke("Hola, ¿cómo estás?")
response.text

'¡Hola! Estoy bien, gracias. ¿Y tú? ¿Cómo estás?'

In [3]:
response = llm.invoke("¿como esta el clima en Morelia Michoacan?")
response.text

'Actualmente en Morelia, Michoacán, el clima es soleado con una temperatura de alrededor de 25 grados Celsius. Se espera que el día continúe soleado y caluroso, con una ligera brisa en la tarde. No se esperan lluvias para el día de hoy.'

In [4]:
response = llm.invoke("Dime los productos que tienes en la tienda")
response.text

'Lo siento, no tengo una lista específica de productos en la tienda ya que soy un asistente virtual y no tengo acceso a esa información. ¿Hay algo más en lo que pueda ayudarte?'

In [15]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filtered by prices")
def get_products():
  """Get the products that the store offers"""
  # Connect with API
  response = requests.get("https://api.escuelajs.co/api/v1/products")
  products = response.json()
  return "".join([f"{product['title']} - {product['price']}" for product in products])


In [16]:
get_products.invoke({})

'Sleek Modern Laptop for Professionals - 97Elegant Golden-Base Stone Top Dining Table - 660Modern Elegance Teal Armchair - 25Elegant Solid Wood Dining Table - 67Modern Minimalist Workstation Setup - 49Modern Ergonomic Office Chair - 71Futuristic Holographic Soccer Cleats - 39Chic Summer Denim Espadrille Sandals - 33Vibrant Pink Classic Sneakers - 84Futuristic Silver and Gold High-Top Sneaker - 68Elegant Patent Leather Peep-Toe Pumps with Gold-Tone Heel - 5300Elegant Purple Leather Loafers - 170Classic Blue Suede Casual Shoes - 39Sleek Futuristic Electric Bicycle - 2200Super Car - 500000New Product - 10user_1766855044647mm - 100jhh - 88Marjory73 - 10'

In [17]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
  """Get the weather of a city"""
  response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
  data = response.json()
  latitude = data["results"][0]["latitude"]
  longitude = data["results"][0]["longitude"]
  response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
  weather_data = response.json()
  current_weather = weather_data["current_weather"]
  return f"The current temperature in {city} is {current_weather['temperature']}°C with wind speed of {current_weather['windspeed']} km/h."

get_weather.invoke({"city": "Morelia"})

'The current temperature in Morelia is 17.2°C with wind speed of 6.1 km/h.'

In [22]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar productos y obtener información sobre el clima en diferentes ciudades.

Tus tools son:
1. get_products: Obtiene la lista de productos disponibles en la tienda junto con sus precios.
2. get_weather: Obtiene el clima actual de una ciudad específica.
"""

messages = [
  ("system", system_prompt),
  ("user", "Hola, ¿qué productos tienes en la tienda?")
]

llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.pretty_print()
response.tool_calls

================================== Ai Message ==================================
Tool Calls:
  get_products (call_gO2lHgeYKGNKDeZZg3qaTpcT)
 Call ID: call_gO2lHgeYKGNKDeZZg3qaTpcT
  Args:


[{'name': 'get_products',
  'args': {},
  'id': 'call_gO2lHgeYKGNKDeZZg3qaTpcT',
  'type': 'tool_call'}]